<a href="https://www.kaggle.com/code/akouaorsot/data-cleaning-british-airways-reviews?scriptVersionId=139307481" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Project 1: Cleaning the reviews of British Airways Passengers between 2016-2023

With data cleaning taking up a considerable chunk of data science workflow, it is critical to know how to assess, improve, and produce data quality. Every subsequent steps from analysis to machine learning modeling hinges on having good data. And so, this project will focus on practicing best practices when it comes to data cleaning to ensure higher quality and usability. 

In that vein, we examined various sources, especially the following three in an attempt to draft a comprehensive map for the cleaning process: 
* [World Bank Data Cleaning Checklist](https://dimewiki.worldbank.org/Checklist:_Data_Cleaning)
* [Mariann Beargie on Medium](https://mariannbea.medium.com/are-you-sure-your-data-is-completely-clean-use-this-checklist-to-help-8cb415a8b0e0)
* [DataCamp Data Cleaning Infographic](https://www.datacamp.com/blog/infographic-data-cleaning-checklist)

Based on the sources above, we modelled our steps and ensure that we could produce a score on the scale of 1 to 10 before and after cleaning.

In [1]:
## Importing Libraries
import os           # operating system
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import category_encoders as ce # Encode Categorical Variables

import matplotlib.pyplot as plt # data visualization

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
## Importing the dataset
paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))

paths

['/kaggle/input/continent2/continents2.csv',
 '/kaggle/input/british-airways-passenger-reviews-2016-2023/British_Airway_Review.csv']

In [3]:
df = pd.read_csv(paths[1])
df.head()

,reviews,date,country,seat_type,recommended,stars,route,type_of_traveller
0,✅ Trip Verified | I had the most fantastic BA...,1st August 2023,Hong Kong,Business Class,yes,5,Heathrow to Las Vegas,Family Leisure
1,✅ Trip Verified | Couldn’t book in online. Ar...,31st July 2023,United Kingdom,Economy Class,no,3,Rome to Heathrow,Solo Leisure
2,✅ Trip Verified | London Heathrow to Mumbai in...,31st July 2023,Iceland,Business Class,yes,3,Gatwick to Venice,Solo Leisure
3,"✅ Trip Verified | Keflavík, Iceland to London ...",31st July 2023,Iceland,Business Class,yes,5,London to Luanda,Couple Leisure
4,✅ Trip Verified | Terrible Experience with Bri...,29th July 2023,Canada,Economy Class,no,5,Denver to Heathrow,Family Leisure


In [4]:
# Overview of the data: 2500 rows, 8 columns, no missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   reviews            2500 non-null   object
 1   date               2500 non-null   object
 2   country            2500 non-null   object
 3   seat_type          2500 non-null   object
 4   recommended        2500 non-null   object
 5   stars              2500 non-null   int64 
 6   route              2500 non-null   object
 7   type_of_traveller  2500 non-null   object
dtypes: int64(1), object(7)
memory usage: 156.4+ KB


In [5]:
# No duplicate values detected
df[df.duplicated()]

,reviews,date,country,seat_type,recommended,stars,route,type_of_traveller


In [6]:
# Column containing reviews in textual form.
for col in df.columns[2:]:
    print(df[col].value_counts())

United Kingdom           1552
United States             300
Canada                     80
Australia                  75
Germany                    53
                         ... 
Saint Kitts and Nevis       1
Botswana                    1
Chile                       1
Senegal                     1
Finland                     1
Name: country, Length: 67, dtype: int64
Economy Class      1363
Business Class      754
Premium Economy     248
First Class         135
Name: seat_type, dtype: int64
no     1620
yes     880
Name: recommended, dtype: int64
3    1137
5     682
1     227
9     227
7     227
Name: stars, dtype: int64
Heathrow to Las Vegas    250
Rome to Heathrow         250
Gatwick to Venice        250
London to Luanda         250
Denver to Heathrow       250
BKK to LHR               250
London to Tampa          250
London to Sydney         250
LHR to CPT               250
BLR to LHR               250
Name: route, dtype: int64
Couple Leisure    1000
Family Leisure     500
Solo Leisu

In [7]:
# Create a different column "trip verified" to seperate them properly
df['trip_verified'] = np.where(df['reviews'].str.contains("Trip Verified"), 1, 0) 
df

,reviews,date,country,seat_type,recommended,stars,route,type_of_traveller,trip_verified
0,✅ Trip Verified | I had the most fantastic BA...,1st August 2023,Hong Kong,Business Class,yes,5,Heathrow to Las Vegas,Family Leisure,1
1,✅ Trip Verified | Couldn’t book in online. Ar...,31st July 2023,United Kingdom,Economy Class,no,3,Rome to Heathrow,Solo Leisure,1
2,✅ Trip Verified | London Heathrow to Mumbai in...,31st July 2023,Iceland,Business Class,yes,3,Gatwick to Venice,Solo Leisure,1
3,"✅ Trip Verified | Keflavík, Iceland to London ...",31st July 2023,Iceland,Business Class,yes,5,London to Luanda,Couple Leisure,1
4,✅ Trip Verified | Terrible Experience with Bri...,29th July 2023,Canada,Economy Class,no,5,Denver to Heathrow,Family Leisure,1
...,...,...,...,...,...,...,...,...,...
2495,Roundtrip with British Airways from Bangkok to...,12th November 2015,Thailand,Business Class,no,7,BKK to LHR,Business,0
2496,Awful customer service. My wife and I have flo...,11th November 2015,United Kingdom,Business Class,no,3,London to Tampa,Couple Leisure,0
2497,I travelled from London to Sydney via Singapor...,10th November 2015,Australia,First Class,yes,5,London to Sydney,Couple Leisure,0
2498,British Airways have just moved Cape Town flig...,10th November 2015,United Kingdom,First Class,yes,3,LHR to CPT,Couple Leisure,0


In [8]:
# Converting 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

In [9]:
# In the airline industry, seat pricing implies a given order. 
# So, we will encode accordingly
ordinal_encoder= ce.OrdinalEncoder(cols=['seat_type', 'stars'],return_df=True,
                           mapping=[
                               {'col':'seat_type',
                                'mapping':{'Economy Class':0,'Premium Economy':1,
                                                'Business Class':2,'First Class':3}},
                               {'col':'stars',
                                'mapping':{1:0, 3:1, 5:2,7:3, 9:4}}])
#Original data
df = ordinal_encoder.fit_transform(df)

# 'recommended' and 'trip_verified' are binary categorical variables so, will be encoded as such
binary_encoder= ce.BinaryEncoder(cols=['recommended', 'trip_verified'],return_df=True)
df = binary_encoder.fit_transform(df)
df

,reviews,date,country,seat_type,recommended_0,recommended_1,stars,route,type_of_traveller,trip_verified_0,trip_verified_1
0,✅ Trip Verified | I had the most fantastic BA...,2023-08-01,Hong Kong,2,0,1,2,Heathrow to Las Vegas,Family Leisure,0,1
1,✅ Trip Verified | Couldn’t book in online. Ar...,2023-07-31,United Kingdom,0,1,0,1,Rome to Heathrow,Solo Leisure,0,1
2,✅ Trip Verified | London Heathrow to Mumbai in...,2023-07-31,Iceland,2,0,1,1,Gatwick to Venice,Solo Leisure,0,1
3,"✅ Trip Verified | Keflavík, Iceland to London ...",2023-07-31,Iceland,2,0,1,2,London to Luanda,Couple Leisure,0,1
4,✅ Trip Verified | Terrible Experience with Bri...,2023-07-29,Canada,0,1,0,2,Denver to Heathrow,Family Leisure,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2495,Roundtrip with British Airways from Bangkok to...,2015-11-12,Thailand,2,1,0,3,BKK to LHR,Business,1,0
2496,Awful customer service. My wife and I have flo...,2015-11-11,United Kingdom,2,1,0,1,London to Tampa,Couple Leisure,1,0
2497,I travelled from London to Sydney via Singapor...,2015-11-10,Australia,3,0,1,2,London to Sydney,Couple Leisure,1,0
2498,British Airways have just moved Cape Town flig...,2015-11-10,United Kingdom,3,0,1,1,LHR to CPT,Couple Leisure,1,0


In [10]:
# The 'country' column is a categorical feature, but with 67 distinct entities
# In that regrd, we will proceed differently

# Though there are 67 countries, 
# the majority of travelers are coming certain regions/sub-regions.
# So, we will enrich our dataset by merging it with the one from 'continent2'
geo_df = pd.read_csv(paths[0], usecols=['name', 'region', 'sub-region'])
geo_df.head()

,name,region,sub-region
0,Afghanistan,Asia,Southern Asia
1,Åland Islands,Europe,Northern Europe
2,Albania,Europe,Southern Europe
3,Algeria,Africa,Northern Africa
4,American Samoa,Oceania,Polynesia


In [11]:
new_df = pd.merge(df, geo_df, how='left', left_on = 'country', right_on='name', )

In [12]:
# We now end up with 5 georgaphic groups for each 
# customer's origin continent
new_df['region'].value_counts()

Europe      1827
Americas     403
Asia         144
Oceania       84
Africa        40
Name: region, dtype: int64

In [13]:
# For more granularity, the sub-region breakdown
# offers 13 groups for each customer 
new_df['sub-region'].value_counts()

Northern Europe                    1604
Northern America                    382
Western Europe                      144
Australia and New Zealand            84
Southern Europe                      55
South-eastern Asia                   50
Eastern Asia                         41
Western Asia                         40
Sub-Saharan Africa                   39
Eastern Europe                       24
Latin America and the Caribbean      21
Southern Asia                        13
Northern Africa                       1
Name: sub-region, dtype: int64